In [151]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from nltk.stem.snowball import RussianStemmer
import string

In [3]:
data = pd.DataFrame.from_csv('data/avito_train.tsv', sep='\t')

In [142]:
data.head(10)

,category,subcategory,title,description,attrs,price,is_proved,is_blocked,phones_cnt,emails_cnt,urls_cnt,close_hours
itemid,,,,,,,,,,,,
10000010,Транспорт,Автомобили с пробегом,"Toyota Sera, 1991",Новая оригинальная линзованая оптика на ксенон...,"{""Год выпуска"":""1991"", ""Тип кузова"":""Купе"", ""П...",150000,NaN,0,0,0,0,0.03
10000025,Услуги,Предложения услуг,Монтаж кровли,Выполняем монтаж кровли фальцевой ^p Тел:8@@P...,"{""Вид услуги"":""Ремонт, строительство""}",0,NaN,0,1,0,0,22.38
10000094,Личные вещи,"Одежда, обувь, аксессуары",Костюм Steilmann,Юбка и топ из панбархата. Под топ трикотажная...,"{""Вид одежды"":""Женская одежда"", ""Предмет одежд...",1500,NaN,0,0,0,0,0.41
10000101,Транспорт,Автомобили с пробегом,"Ford Focus, 2011","Автомобиль в отличном техническом состоянии, в...","{""Марка"":""Ford"", ""Модель"":""Focus"", ""Год выпуск...",365000,NaN,0,0,0,0,8.87
10000132,Транспорт,Запчасти и аксессуары,Турбина 3.0 Bar,Продам турбину на двигатель V-6 . V-8 и мощнее...,"{""Вид товара"":""Запчасти"", ""Тип товара"":""Для ав...",5000,NaN,0,0,0,0,11.82
10000152,Транспорт,Автомобили с пробегом,"ВАЗ 2115 Samara, 2005",Автомабиль вхорошем состаянием НЕ ГНЕЛАЯ борт ...,"{""Марка"":""ВАЗ (LADA)"", ""Модель"":""2115 Samara"",...",0,NaN,0,1,0,0,22.55
10000160,Транспорт,Автомобили с пробегом,"ВАЗ 2105, 1998",машина в хорошем состояние! кузов родной салон...,"{""Марка"":""ВАЗ (LADA)"", ""Модель"":""2105"", ""Год в...",48000,NaN,0,0,0,0,9.02
10000224,Недвижимость,Комнаты,"Комната 12 м² в 3-к, 3/14 эт.",Сдам комнату на длительный срок. Желательно де...,"{""Тип объявления"":""Сдам"", ""Срок аренды"":""На дл...",5000,NaN,0,0,0,0,0.74
10000243,Недвижимость,"Дома, дачи, коттеджи",Дом 42 м² на участке 7 сот.,Продается хороший 2х эт. Дом в пос.Победилово ...,"{""Тип объявления"":""Продам"", ""Вид объекта"":""Дом...",490000,NaN,0,1,0,1,18.63


In [116]:
train = data[:10000]
data.shape

(3995803, 12)

In [117]:
# просматриваем информацию в колонках
for column in train.columns:
    print("{: <20} {:} {: >10}".format(column, train[column].dtype, len(train[column].unique())))

category             object         10
subcategory          object         53
title                object       8542
description          object       9842
attrs                object       4405
price                int64       1344
is_proved            float64          3
is_blocked           int64          2
phones_cnt           int64          8
emails_cnt           int64          3
urls_cnt             int64          6
close_hours          float64       2124


In [121]:
# по таблице определяем категориальные string колонки
cat_cols = ['category','subcategory']

In [124]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
# from sklearn.feature_extraction import DictVectorizer

enc = LabelEncoder()
train[cat_cols] = train[cat_cols].apply(lambda x: pd.factorize(x)[0])

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:2440: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [170]:
# стемминг и знаки пунктуации
stemmer = RussianStemmer()
exclude = set(string.punctuation)
exclude = exclude | set(map(str,[1, 2, 3, 4, 5, 6, 7, 8, 9, 0]))

# разделение по словам без знаков пунктуации и цифр
def clear(title):
    temp = ''.join([i for i in str(title) if i not in exclude])
    temp = temp.split()
    temp = [stemmer.stem(i).lower() for i in temp]
    temp = [i for i in temp if len(i) > 2]
    return temp
train['title'].apply(clear)
# clear("Турбина 3.0 Bar")

itemid
10000010                                         [toyot, ser]
10000025                                      [монтаж, кровл]
10000094                                   [костюм, steilman]
10000101                                        [ford, focus]
10000132                                        [турбин, bar]
10000152                                         [ваз, samar]
10000160                                                [ваз]
10000224                                             [комнат]
10000243                                   [дом, участк, сот]
10000247                                   [подголовник, ваз]
10000255                                         [daewo, nex]
10000263                                   [дом, участк, сот]
10000287            [appl, iphon, black, simfr, европ, идеал]
10000290                [дизайн, интерьер, комплексн, отделк]
10000299     [костюм, didriksons, boardma, размер, краг, шап]
10000309                                            [квартир]
1